# CORE Memory Client Demo

This notebook demonstrates the functionality of the updated `CoreMemoryClient` class, which provides comprehensive access to CORE Memory APIs including ingestion, search, logging, and space management.

## Features Demonstrated

**Working (MCP Protocol):**
- ✅ Space management (list, create, details)
- ✅ Data ingestion (via memory_ingest tool)
- ✅ Knowledge graph search (via memory_search tool)

**Pending (Direct API Endpoints):**
- ⏳ Space updates
- ⏳ Ingestion log management
- ⏳ Individual and bulk log deletion
- ⏳ Episode facts retrieval
- ⏳ User profile access

*Note: Some features return placeholder data until direct API endpoints are activated.*

In [ ]:
# Import required libraries
import os
import sys
import json
from pathlib import Path
from pprint import pprint

# Change to project root directory (same setup as mcode_translator.ipynb)
current_dir = Path.cwd()
if current_dir.name == 'examples':
    project_root = current_dir.parent
    os.chdir(project_root)
    print(f"📁 Changed working directory to: {project_root}")
else:
    project_root = current_dir

# Add project root to Python path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print("✅ Added project root to Python path")

# Now we can import from src
from src.utils.core_memory_client import CoreMemoryClient

# Set your CORE Memory API key
# You can set this in your environment or .env file as COREAI_API_KEY
# For demo purposes, we'll assume it's set
print("Make sure COREAI_API_KEY is set in your environment")
print(f"📍 Current working directory: {Path.cwd()}")

# Helper function to pretty print JSON responses
def print_json(data, title="Response"):
    print(f"\n{title}:")
    print(json.dumps(data, indent=2))

In [ ]:
# Initialize the client
# The client will automatically load configuration from src/config/core_memory_config.json
try:
    client = CoreMemoryClient()
    print("✅ CORE Memory Client initialized successfully")
    print("📋 Client loaded configuration:")
    print(f"   - API Base URL: {client.url.split('?')[0]}")
    print(f"   - Timeout: {client.timeout} seconds")
except Exception as e:
    print(f"❌ Failed to initialize CORE Memory Client: {e}")
    print("\n🔧 Troubleshooting:")
    print("   1. Make sure COREAI_API_KEY is set in your environment")
    print("   2. Check that src/config/core_memory_config.json exists")
    print("   3. Verify the config file has valid JSON")
    raise  # Re-raise to stop execution

## Space Management

Let's start by exploring and managing spaces in CORE Memory.

In [ ]:
# List all available spaces
spaces = client.get_spaces()
print(f"📂 Found {len(spaces)} spaces:")
for space in spaces:
    print(f"   - {space.get('name')} (ID: {space.get('id')})")

In [ ]:
# Search for spaces
demo_spaces = client.get_spaces(query="demo")
print(f"🔍 Found {len(demo_spaces)} spaces matching 'demo':")
for space in demo_spaces:
    print(f"   - {space.get('name')}")

In [ ]:
# Create a demo space for testing
demo_space_id = client.create_space(
    name="Demo Space",
    description="A temporary space for demonstrating CORE Memory functionality"
)
print(f"✨ Created demo space with ID: {demo_space_id}")

In [ ]:
# Get space details
space_details = client.get_space_details(demo_space_id)
print("📋 Space details:")
print(f"   Name: {space_details.get('space').get('name')}")
print(f"   Description: {space_details.get('space').get('description')}")
print(f"   Created: {space_details.get('space').get('createdAt')}")

In [ ]:
# Update space information (may not be supported by current API)
try:
    updated_space = client.update_space(
        space_id=demo_space_id,
        name="Updated Demo Space",
        description="An updated demo space for CORE Memory functionality"
    )
    print(f"✅ Updated space: {updated_space.get('name')}")
except Exception as e:
    print(f"⚠️  Space updates may not be supported: {e}")
    print("   Continuing with demo...")

## Data Ingestion

Now let's ingest some sample data with different priorities and sources.

In [ ]:
# Ingest sample data with different priorities and sources
sample_data = [
    {
        "message": "Patient John Doe diagnosed with Stage II breast cancer",
        "source": "demo_clinical",
        "priority": "high",
        "tags": ["patient", "diagnosis", "breast_cancer"]
    },
    {
        "message": "Clinical trial NCT12345678 for metastatic breast cancer",
        "source": "demo_trials",
        "priority": "normal",
        "tags": ["clinical_trial", "breast_cancer"]
    },
    {
        "message": "Research paper on immunotherapy for lung cancer",
        "source": "demo_research",
        "priority": "low",
        "tags": ["research", "lung_cancer", "immunotherapy"]
    },
    {
        "message": "Patient Jane Smith completed chemotherapy cycle",
        "source": "demo_clinical",
        "priority": "normal",
        "tags": ["patient", "treatment", "chemotherapy"]
    }
]

ingestion_results = []
for data in sample_data:
    result = client.ingest(
        message=data["message"],
        space_id=demo_space_id,
        source=data["source"],
        priority=data["priority"],
        tags=data["tags"]
    )
    ingestion_results.append(result)
    print(f"✅ Ingested: {data['message'][:50]}...")
    print(f"   Result ID: {result.get('id', 'N/A')}")
    print()

## Search Functionality

Let's search the knowledge graph for ingested data.

In [ ]:
# Search for breast cancer related content
search_results = client.search(
    query="breast cancer",
    space_id=demo_space_id,
    limit=10
)
print("🔍 Search results for 'breast cancer':")
print(f"   Episodes found: {len(search_results.get('episodes', []))}")
print(f"   Facts found: {len(search_results.get('facts', []))}")

# Display episodes
episodes = search_results.get('episodes', [])[:3]
if episodes:
    print("📄 Sample episodes:")
    for i, episode in enumerate(episodes, 1):
        if isinstance(episode, dict):
            content = episode.get('content', '')[:100]
        else:
            content = str(episode)[:100]
        print(f"   {i}. {content}...")
else:
    print("   No episodes found")

# Show full search results in pretty format
print_json(search_results, "📊 Complete Search Results")

In [ ]:
# Search for patient-related content
patient_results = client.search(
    query="patient",
    space_id=demo_space_id,
    include_episodes=True,
    include_facts=False
)
print(f"👥 Found {len(patient_results.get('episodes', []))} patient-related episodes")

## Ingestion Log Management (PENDING)

Ingestion log management features are currently pending API activation.
These methods return placeholder data for demonstration purposes.

In [ ]:
# Get ingestion logs for our demo space
logs = client.get_ingestion_logs(space_id=demo_space_id, limit=20)
print(f"📋 Found {len(logs)} ingestion logs")

# Display log summary
for log in logs[:5]:  # Show first 5
    print(f"📝 Log ID: {log.get('id')}")
    print(f"   Source: {log.get('source')}")
    print(f"   Status: {log.get('status')}")
    print(f"   Message: {log.get('message', '')[:50]}...")
    print()

In [ ]:
# Get detailed information about a specific log
if logs:
    specific_log = client.get_specific_log(logs[0]['id'])
    print_json(specific_log, "🔍 Detailed Log Information")
else:
    print("No logs available to examine")

## Individual Log Deletion

Demonstrate deleting logs one by one.

In [ ]:
# Delete individual logs from demo_research source
research_logs = [log for log in logs if log.get('source') == 'demo_research']
print(f"🎯 Found {len(research_logs)} logs from demo_research source")

for log in research_logs:
    try:
        delete_result = client.delete_log_entry(log['id'])
        print(f"🗑️  Deleted log {log['id']}: Success")
    except Exception as e:
        print(f"❌ Failed to delete log {log['id']}: {e}")

## Bulk Log Deletion by Source

Demonstrate deleting all logs from a specific source in bulk.

In [ ]:
# Function to delete all logs from a specific source
def delete_logs_by_source(source_name: str, space_id: Optional[str] = None):
    """
    Delete all ingestion logs from a specific source.
    
    Args:
        source_name: The source identifier to filter logs
        space_id: Optional space ID to filter logs
    
    Returns:
        Number of logs deleted
    """
    # Get all logs for the source
    all_logs = client.get_ingestion_logs(space_id=space_id, limit=1000)
    source_logs = [log for log in all_logs if log.get('source') == source_name]
    
    deleted_count = 0
    for log in source_logs:
        try:
            client.delete_log_entry(log['id'])
            deleted_count += 1
            print(f"🗑️  Deleted log {log['id']} from source '{source_name}'")
        except Exception as e:
            print(f"❌ Failed to delete log {log['id']}: {e}")
    
    return deleted_count

# Delete all logs from demo_clinical source
deleted_count = delete_logs_by_source('demo_clinical', demo_space_id)
print(f"\n📊 Total logs deleted from demo_clinical: {deleted_count}")

In [ ]:
# Delete remaining logs from demo_trials source
deleted_count_trials = delete_logs_by_source('demo_trials', demo_space_id)
print(f"\n📊 Total logs deleted from demo_trials: {deleted_count_trials}")

## Episode Facts (PENDING)

Episode facts retrieval is currently pending API activation.
This method returns placeholder data for demonstration purposes.

In [ ]:
# Get episode facts from our demo space
facts = client.get_episode_facts(space_id=demo_space_id, limit=10)
print(f"🧠 Found {len(facts)} episode facts")

# Display sample facts
for fact in facts[:3]:
    print(f"💡 Fact: {fact.get('content', '')[:100]}...")
    print(f"   Episode ID: {fact.get('episode_id')}")
    print(f"   Confidence: {fact.get('confidence')}")
    print()

## User Profile (PENDING)

User profile access is currently pending API activation.
This method returns placeholder data for demonstration purposes.

In [ ]:
# Get user profile
profile = client.get_user_profile()
print_json(profile, "👤 User Profile")

## Cleanup

Clean up the demo space.

In [ ]:
# Note: Space deletion might not be implemented in the API
# For cleanup, you could manually delete the demo space through the CORE web interface
print("🎉 Demo completed! Check the CORE web interface to verify the operations.")
print(f"🏷️  Demo space ID: {demo_space_id}")